# Looking at the dataset structure
Since the dataset was not created by us, but has been obtained by an external source, before start to model the data, it is necessary understand all the variables contained the file and do some checks in order to detect possible errors or outliers

In [1]:
import csv
import pandas as pd
import numpy as np

In [2]:
beer = pd.read_csv ('beer_clean.csv')

## How many data do we have?, How is our dataset structure   
There are 104 styles of beer, and 5.840  brewers companies which supply 56.845 different beers. The Id is an unique identify for each beer, so if there are more Ids (66.036) than names of beers means that we have cases with same style, same supplier and same name of beer but labeled with different Id

#### Hierarchy   
1.Style --> 104   
2.BrewerId --> 5840   
3.Names --> 56845   
4.Id --> 66036

In [3]:
# total customers
customers=beer['profile_name'].nunique()
customers

33381

In [4]:
# total beers by style
beer_style=beer['style'].nunique()
beer_style

104

In [5]:
# total beers by Id
beer_Id=beer['Id'].nunique()
beer_Id

66036

In [6]:
# total beers by brewerId
beer_brewerId=beer['brewerId'].nunique()
beer_brewerId

5840

In [7]:
# total beers by names
beer_name=beer['names'].nunique()
beer_name

56845

## Question 1: 
### We have less names than Ids. Are there beers with the same name, style and same brewerId, but different Id? 
As we have seen, there are 56.845 names of beers and 66.036 Ids. There must be an explanation for those names having more than one Id.   
Grouping by the names, we can see how there are names of beer with more than 100 different ID, e.g Oktoberfest. Instead of one name = one ID 


In [8]:
brand_by_name=beer.groupby('names')['Id'].nunique()
brand_by_name.sort_values(ascending = False).head(10)

names
 Oktoberfest       199
 Pale Ale          198
 Oatmeal Stout     161
 IPA               147
 Hefeweizen        143
 India Pale Ale    140
 Porter            107
 Maibock           101
 Imperial Stout     95
 Amber Ale          86
Name: Id, dtype: int64

### Eg,name of beer = Oktoberfest has 199 Ids. Why?

In [9]:
## the beer called Oktoberfest has 5 different styles
brand_study=beer.groupby(['names']).get_group(' Oktoberfest')['style'].nunique()
brand_study


5

In [10]:
brand_study=beer.groupby(['names']).get_group(' Oktoberfest')
brand_by_name2=brand_study.groupby(['style'])['Id'].nunique()
brand_by_name2

style
 American Amber / Red Ale        1
 American Amber / Red Lager      1
 American Dark Wheat Ale         1
 Märzen / Oktoberfest          194
 Vienna Lager                    2
Name: Id, dtype: int64

### Checking the Märzen / Oktoberfest style which has 194 Ids
- **It could be that the Id changes within the same beer name and style attending to the ABV variable**   
- Same name, same style, differents Ids (probably the ABV has something to do with it)   
- Is it possible that differents brewer company have the same name of beer?   

In [11]:
brand_by_name3=brand_study.groupby(['style','ABV'])['Id'].nunique()
brand_by_name3

style                        ABV  
 American Amber / Red Ale    99.00      1
 American Amber / Red Lager  5.40       1
 American Dark Wheat Ale     5.20       1
 Märzen / Oktoberfest        3.80       1
                             4.30       1
                             4.80       1
                             4.90       3
                             5.00       7
                             5.10       1
                             5.20       6
                             5.30       3
                             5.40       4
                             5.50       8
                             5.60       3
                             5.70       4
                             5.75       1
                             5.80       8
                             5.90       2
                             6.00      15
                             6.20       3
                             6.25       1
                             6.40       1
                             6.50       3

###  Ids are related with the ABV
- Most of the Ids are concentrate on the variable ABV = 99 (we recoded those null fields with a 99 code). This reaffirms the theory that one beer with the same style and the name, can  have different Ids attending to its alcoholic graduation.  
- Let's check now, the  beer that has 6 degrees and 15 differents Ids. By doing a few filters, we can verify that these beers belong to 15 different breweries and therefore we can say that our data are correct (they have sense)

In [12]:
marzen=beer[(beer['style'] == " Märzen / Oktoberfest") & (beer['names'] == " Oktoberfest")  & (beer['ABV'] == 6)]
marzen2=marzen.groupby(['brewerId'])['Id'].nunique()
marzen2

brewerId
144      1
395      1
959      1
1683     1
2310     1
2454     1
2599     1
3925     1
4426     1
10567    1
11936    1
15205    1
21683    1
22035    1
24791    1
Name: Id, dtype: int64

# Most popular beers
**American IPA** with 3.167 different beers, and the **American Pale Ale** with 2.886, are the styles of beer more offered by the brewers.   
The **IPA style**, following by the **Imperial IPA style**, are the most rated/consumed by the customers (with 12.479 and 10.977 ratings respectively)

In [13]:
# Num of beers (brand/names) broup by style
brand_by_style=beer.groupby('style')['names'].nunique()
brand_by_style.sort_values(ascending = False).head()

style
 American IPA                3167
 American Pale Ale (APA)     2886
 American Amber / Red Ale    2230
 American Porter             1839
 Fruit / Vegetable Beer      1603
Name: names, dtype: int64

In [14]:
brand_by_style.sort_values(ascending = True).head()

style
 Faro                               19
 Bière de Champagne / Bière Brut    20
 Sahti                              25
 Gose                               27
 Kvass                              43
Name: names, dtype: int64

In [15]:
# Beers most rated by style
most_rated_style=beer.groupby('style')['profile_name'].nunique()
most_rated_style.sort_values(ascending = False).head()

style
 American IPA                      12479
 American Double / Imperial IPA    10977
 American Pale Ale (APA)            9252
 Russian Imperial Stout             8990
 American Porter                    8464
Name: profile_name, dtype: int64

In [16]:
customerPreferences2=beer.groupby('names')['profile_name'].nunique()
customerPreferences2.sort_values(ascending = False).head()

names
 90 Minute IPA                          3206
 Old Rasputin Russian Imperial Stout    3038
 Sierra Nevada Celebration Ale          2929
 Stone Ruination IPA                    2644
 Arrogant Bastard Ale                   2633
Name: profile_name, dtype: int64

# Customer preferences
## Question 2:    
### Is it possible that there are customers who have rated more than 100 beers in one day?
- It could be that they make annotations and then rate all the beers that they tested at once
- It could be a profile_name of a shop, or it can corresponds to a event.   
- Also we can see how there is a profile name labeled as BeerAdvocate which is the name of the website (from where we took the data). So it could be a good idea to take it out.   

Let's check the data in more detail

In [17]:
# Customer preferences 
customers=beer.groupby(['profile_name', 'year', 'month', 'day'])['names'].count()
customers.nlargest(30)

profile_name    year  month  day
 RblWthACoz     2007  12     30     361
                             31     257
                2008  1      1      247
 LilBeerDoctor  2010  12     23     217
 magictrokini   2008  12     10     215
                             5      214
 JDV            2008  4      6      176
 changeup45     2008  8      24     173
 yesyouam       2008  12     20     169
 JDV            2008  6      30     168
 magictrokini   2008  12     16     165
 changeup45     2008  8      23     157
 thekevlarkid   2008  8      20     153
 BeerAdvocate   2011  1      11     146
 LilBeerDoctor  2009  7      11     146
                2008  12     23     141
                             20     132
 JDV            2008  3      15     126
 Thorpe429      2010  10     24     122
                      11     14     122
 LilBeerDoctor  2009  12     19     115
 BeerAdvocate   2010  9      20     104
 BMan1113VR     2008  2      7      102
 grub           2011  2      4       97
 magict

- The customer who rated more beers in one day is _"RblWthACoz"_ .Another one that rated a lot of beers in different days is called _"JDV"_.   
- We can check the **customer comments to see if they are different  over all their ratings** (if they are quite similar or equal , it could means that they are not from a real customer)
- In order to have an idea of if their **ratings are real ones** or look like they **belong to a store** or they comming from a kind of **automatic scoring**   
- If we have into account the **ratio between the ratings per day and the total ratings of each customer**, we can check if the higher rating days are a normal behavior of the customer or an outlier

### RblWthACoz comments: it seems that they are differents

In [18]:
pd.options.display.max_colwidth = 1000
customer1=beer.loc[beer['profile_name'] == ' RblWthACoz',['text']]
customer1

,text
2484,""" Pours a clear, but darker ruby hued brown. Thin light tan head. Flavor is sweet, and too much so I have to say. Calms down a bit over time, but the memory of it sticks with me. Fruit tones and a tart bitterness ride on the backside of things. Feel is slick on liquid and the carbonation is slightly subdued. This is an okay beer. As it warms it's initially abrasive qualities seem to calm down a bit and I enjoy it more than I thought I would.\t\t"""
2485,""" Quite the schnazzy label we've got here. \t\tPours a foggy orange gold with a decent off-white head. Nose is like fruit that's been sitting out too long. Wow. Flavor is clearly smoked. This is like a rauchbier, yet Belgian in style. Defined meat tones and smoky as well. As I drink more of it, I see where it comes from on the nose. Besides everything else so far, there is a clear sweet edge with fruit tones as well. As time goes on the rauch qualities fade away and a sweet tone really sets in. Yet occasionally the rauch character hits me in the back of the mouth as an aftertaste ever so subtly. Feel is slick and smooth, yet a bit thick, on liquid. Carbonation is middle of the road. Well this was an interesting beer. I definitely suggest you try it if you have the chance.\t\t"""
2487,""" Pours a foggy orange with a brown touch and a thicker light head. Nose is like citrus rinds and concentrated orange as well. Flavor is intense orange, almost like the flavor of Tang. A bit too much IMO. Feel has a fuller carbonation. Liquid is about right. This beer just doesn't work for me. Way too much tang and unbalanced.\t\t"""
2488,""" Pours a foggy orange gold with a decent sized crme head. Nose is quite sweet, but not overpowering. Definite citrus as well. Flavor is interestingly complex. Sweet tones are firmly there with numerous fruit layers. Feel is a bit thicker on liquid. Carbonation is full enough, though not abrasive. I like this beer. Quite tasty and a pleasure to drink.\t\t"""
2989,Pours a slightly hazed orange amber. Nose is a lot of dough and spices. Flavor is tangy with citrus. Nice level of sweetness. Spices are backing it up. Smooth and nice really. Much better than I was expecting. Feel is smooth on liquid. But the one detractor is the carbonation. A bit too much IMO. However. This is still a pretty tasty brew.\t\t
3229,""" Pours a clear very dark amber with brown and reddish tints. Minimal tannish head doesn't really stay long, but leaves decent lacing on the way down. Nose is super floral hops. Strong and unrelenting. Flavor is very strong. Acidic almost at first. Firm woody edges. Very, very earthy. Feel is sticky on liquid and the carbonation is good. Surprisingly, this ends up being a pretty drinkable DIPA as I tore through the bottle and was finished well before I wished. A nice DIPA, though a bit heavy on the robust side.\t\t"""
3874,""" Pours a clear amber with a full creme head that sticks around and leaves nice lace on the way down. Nose is hoppy, of course, but has a soapy edge. Overall fragrances are not very powerful and carry very minor pine character. Flavor is smooth. Balanced bitter tones with noticeable leafy and woody hop tones, though not firm or anywhere near overpowering. Seems like some alcohol is there as well. Feel is nice. Pretty full carbonation on a slick liquid feel. This is a nice drinker. More woody than most other things. I think this on cask would be interesting.\t\t"""
5568,""" Pours pitch black and thick with a smaller dark tan head that forms and falls to nothing. No lacing. Malty tones on the nose. Extremely slight amount of roasted edge to it. Flavor is extremely smooth. Unbelievably smooth. Thick malts accented by mild oat. Sweet tone almost like a molasses that's more smooth and sweet than thick and bitter with a biting edge. Man, oh man. Very mild amount of alcohol biting on the back side. On the aftertaste I feel it burning just a slight bit as well. Feel is thick on liquid and carries a low, but smooth enough carbon

### JDV it seems that they are differents

In [19]:
pd.options.display.max_colwidth = 1000
customer2=beer.loc[beer['profile_name'] == ' JDV',['text']]
customer2

,text
605,""" Can from Beerzack. Thanks!\tPours Semi-clear reddish amber witha big lasting chunky whitish head that leaves lots of lacing. Smell is sweet fruity, fresh and hoppy. Very nice. Taste is smooth with fruity caramel and fresh citrusy, and sweet orange flavored hops and bitter citrus rind. Nice and strongly bittered which lasts in the finish but with a light lingering sweetness too. Very good IPA.\t\t"""
926,""" Bottle shared with Fly, Dewbrewer, Humulone_Red and others. Thanks guys!\tPours a glowing orange with no head. Funky nose reminiscent of gooey band-aids. Odd taste with sweet guaze like flavor. No carbonation on it either, and this one just wasn't for me.\t\t"""
1104,""" Pours pale yellow with almost no head. Smell is sweet with corn and maltiness in the taste. Very carbonated, and light, obviously, easy to drink, and tastes decent enough. Nothing spectacular, but I don't get all the really low ratings. This wasn't supposed to be a big bold beer that turned out light. Rating it for what it is, this is good for the style.\t\t"""
3004,""" Shared at tasting, courtesy of Beervirgin72. Thank you! \tPours a rich cloudy goldish color with a light whitish head. Smell is yeasty and wheaty, but without much clove or spiced characteristic. Taste is wheat, honey, yeast, and lemon and feels more like a cross between an American Wheat and a German Hefeweizen, leaning on the American Wheat side, I thought. Very crisp, and drinkable, with a milder characteristic yeast flavor than most hefs. Nice beer!\t\t"""
3208,""" Bottle from Miketd. Thanks for this one, Mike! Pours rich dark amber with a big creamy beige head in a tulip. Smell is fruity, sweet, and light piney hops and a bit of alcohol, and some caramel sweetness. Taste is creamy, rich and balanced for the style with an emphasis on hops of course. Nicely hopped but lightly caramelly sweet on the backend with a bit of an alcohol presence in the flavor. Very nice, and drinkable DIPA I thought, and a very well made beer.\t\t"""
3869,""" Big thanks to miketd for the bottle.-\tRich but transluscent copper with a lasting off white foamy head. Smell is not as hoppy as expected initially, and more of toffee sweetness, as in an ESB, plus a touch of floral hoppiness. Somewhat of a cantaloupe melony nose after it warms too. Carbonated body, with a more malty than average taste for an IPA, with some light hops and a creaminess to the body despite the slightly bubbly carbonation. A touch of light pepperiness to it as well. Nice IPA overall.\t\t"""
5526,""" Big thanks to Miketd for the bottle!\tPours a solid dark black with a nice sized creamy but somewhat short lived tan head into a snifter. Smell was big and of oatmeal sweetness, dark roasted malt and I got a big fire roasted marshmallow gooeyness out of it as well. Taste was also fantastic, very nice roasty character with oatmeal creaminess and chewiness making an obvious presence. Very full mouthfeel, and on the sweeter side of an imperial stout, which I tend to like. I got a big marshmallow like character in the taste as well and overall this was an amazing beer. The hint of alcohol that shows as it warms was pretty mild and unobtrusive.\t\t"""
5867,""" Bottle courtesy of miketd. Thanks man! Pours nice and black with a lasting, creamy tan head. Smell is dusty bittersweet chocolate and dark cherry fruitiness. Taste is bittersweet chocolate and dark fruitiness. Leaning on the sweet side with a chocolate covered cherry flavor. Creamy soft carbonation and a semi-rich body. Very easy drinking. Really nice.\t\t"""
7066,""" Mahogany dark amber color with lacing and white head. Smell of dark malt mostly. A touch of bitterness, toasty malt, but a bit watery though, and light coffee flavors even. Different for a bock, I thought, and ok, but nothing to get too excited about.\t\t"""
7067,""" Pale hazy yellowish pour. Taste was what you'd expect from a hefeweizen to me, and was clovey and spiced. I thought it was boring, and average, and not d

### Let's check the ratios 

In [20]:
totales=beer.groupby(['profile_name'])['names'].count()
total_by_customer=totales.to_frame('total_by_customer') 
total_by_customer.reset_index(inplace=True)
total_by_customer.head()

,profile_name,total_by_customer
0,007,5
1,00766,2
2,01001111,5
3,0110x011,139
4,01121987,29


In [22]:
totales2=beer.groupby(['profile_name', 'year', 'month', 'day'])['names'].count()
total_by_day=totales2.to_frame('total_customer_day') 
total_by_day.reset_index(inplace=True)
total_by_day.head()

,profile_name,year,month,day,total_customer_day
0,007,2008,9,4,1
1,007,2010,9,30,1
2,007,2010,10,12,1
3,007,2010,10,27,1
4,007,2010,12,8,1


In [23]:
customer_percent=pd.merge(total_by_customer, total_by_day, on='profile_name', how='outer')
customer_percent['rating_percent'] = ((customer_percent['total_customer_day'] / customer_percent['total_by_customer'])*100).round(decimals=2)
customer_percent.head()

,profile_name,total_by_customer,year,month,day,total_customer_day,rating_percent
0,007,5,2008,9,4,1,20.0
1,007,5,2010,9,30,1,20.0
2,007,5,2010,10,12,1,20.0
3,007,5,2010,10,27,1,20.0
4,007,5,2010,12,8,1,20.0


### There are customers that one day rated more than their 40% of total ratings

In [24]:
customer_percent_grouped=customer_percent.groupby(['profile_name', 'year', 'month', 'day'])
result_customer_percent_grouped=customer_percent_grouped['total_customer_day','total_by_customer','rating_percent'].max()
result_customer_percent_grouped.reset_index(inplace=True)

only_few_beers_rated=result_customer_percent_grouped.loc[(result_customer_percent_grouped['rating_percent'] >=40) & (result_customer_percent_grouped['rating_percent'] <100) & (result_customer_percent_grouped['total_by_customer']>120)]
only_few_beers_rated

,profile_name,year,month,day,total_customer_day,total_by_customer,rating_percent
22936,BMan1113VR,2008,2,7,102,133,76.69
278034,RSRIZZO,2008,11,23,57,140,40.71
298964,Samp01,2010,11,24,69,165,41.82
356619,Vendetta,2008,12,7,86,193,44.56
543416,femALEist,2010,3,25,88,152,57.89
630580,kdoc8,2007,3,13,68,157,43.31


### There are  10.437 customers that have rated a maximun of one beer by day, and the total beers rated by those customers are 30.674. It represents the 91,9 % of the total of beers rated in the dataset (33.381)

In [26]:
total_by_customer.head()

,profile_name,total_by_customer
0,007,5
1,00766,2
2,01001111,5
3,0110x011,139
4,01121987,29


In [27]:
num_ratings_customer=total_by_customer.groupby(['total_by_customer'])['profile_name'].nunique()
num_ratings_customer.nlargest(6)

total_by_customer
1    10437
2     4110
3     2424
4     1601
5     1245
6      932
Name: profile_name, dtype: int64

In [28]:
only_one_beer=total_by_day.loc[total_by_day['total_customer_day'] ==1]
only_one_beer.groupby(['profile_name'])['profile_name'].nunique().count()

30674

In [29]:
final_customers=only_one_beer.drop_duplicates(subset='profile_name', keep='first', inplace=False)

In [30]:
final_customers.head()

,profile_name,year,month,day,total_customer_day
0,007,2008,9,4,1
5,00766,2011,10,11,1
8,01001111,2012,1,2,1
11,0110x011,2008,3,15,1
129,01121987,2009,4,8,1


In [31]:
final_customers=final_customers.filter(items=['profile_name'])

In [32]:
final_dataset=pd.merge(final_customers, beer, on='profile_name', how='left')
final_dataset.groupby(['profile_name'])['profile_name'].nunique().count()

30674

In [33]:
final_dataset.head()

,profile_name,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,names,Id,brewerId,ABV,style,appearance,aroma,palate,taste,overall,time,text,time2,day,month,year
0,007,985797,985797,985797,Fraoch Heather Ale,245,12142,5.0,Scottish Gruit / Ancient Herbed Ale,3.5,2.5,3.5,4.0,4.5,1220542724,""" First tried this as part of a mix case several years ago and had been wanting more ever since. Found it by the case and eagerly bought it.\t\tAppearance - pale to amber, slightly cloudy, weak head\t\tSmell - faint floral or herbal notes\t\tTaste - Again, floral and herbal notes, lacking the typical bitter hops finish\t\tMouthfeel - smooth, almost watery\t\tDrinkability - I think this is a must-try beer, if only for it's unique ingredients and taste. I will enjoy drinking it while I have it, but wouldn't make it a staple.\t\t""",2008-09-04,4,9,2008
1,007,1539730,1539730,1539730,Saison Du BUFF,61351,345,6.8,Saison / Farmhouse Ale,4.0,3.5,4.5,3.5,4.5,1285854092,""" Poured from bottle into pint glass. \t\tHazy golden color with 1-2 finger head that lingered.\t\tAroma was floral in character and more powerful than other saisons.\t\tVarious spice flavors followed by mild hop bitterness made this very enjoyable to drink. The floral aromas, however, make this a brew that I could enjoy in quality, but not in quantity.\t\tWell done.\t\t""",2010-09-30,30,9,2010
2,007,1544437,1544437,1544437,HopDevil Ale,1005,345,6.7,American IPA,4.5,5.0,4.5,5.0,5.0,1286891828,""" A- dark amber, lasting head, delicate ephemeral lacing\tS - piney, citrusy \tT - bitter...no sweet...no spicy...no, wait bitter again...\tM - smooth, balanced carbonation, clean finish\tD - 5.0 exceptional, never disappointing\t\tWith apologies to Sir Arthur Conan Doyle...\t\tTo 007, she was always the beer. I have seldom heard him mention her under any other name. In his eyes she eclipses and predominates the whole of her style. It was not that he felt any emotion akin to love for HopDevil. All emotions,and that one particularly, were abhorrent to his cold, precise but admirably balanced mind. He was, I take it, the most perfect reasoning and observing machine that the world has seen, but as a beer-lover he would have placed himself in a false position. He never spoke of the softer passions, save with a gibe and a sneer.\t\tThey were admirable things for the observer -- excellent for drawing the veil from beer's motives and actions. But for the trained reasoner to admit such int...",2010-10-12,12,10,2010
3,007,1545868,1545868,1545868,Helios (V Saison),47776,345,7.5,Saison / Farmhouse Ale,4.0,3.5,3.5,4.0,4.0,1291840287,""" A. Hazy orange, little head retention but nice lacing\tS. Funky citrus\tT. Lemony malt/hops balance\tM. balanced carbonation, creamy finish\tD. at 7.5% it's a bit heavy, but I could drink this all summer long. Sorry I didn't find it until the end of the season.\t\t""",2010-12-08,8,12,2010
4,007,1549890,1549890,1549890,Victory Lager,1002,345,4.8,Munich Helles Lager,3.0,3.5,3.5,4.0,4.0,1288210632,""" Victory Lager was my introduction to craft beer in the mid-90s. Back in the day, the Victory brewpub was more low-key and the draft selection was limited to five or six flagships and a seasonal, compared to 15+ today. The Lager had such big flavor and aroma compared to the macro-brews I was used to (and didn't particularly like).\t\tI've moved on to other styles, but recently revisited my first. It poured golden and clear with little head and slight effervescence. Clean sweet aroma and crisp, clean malt flavor. On the watery side compared to other styles, but fine for a lager. Overall, very drinkable.\t\t""",2010-10-27,27,10,2010


In [34]:
final_dataset.to_csv('beer_prueba1.csv')